# BOOK RECOMMENDER

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
books = pd.read_csv('Books.csv.zip')
users = pd.read_csv('Users.csv.zip')
ratings = pd.read_csv('Ratings.csv.zip')

C:\Users\navee\AppData\Local\Temp\ipykernel_25848\768372.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv.zip')


In [3]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L',]]

In [4]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
x = ratings['User-ID'].value_counts()>200

In [6]:
x

User-ID
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [7]:
y = x[x].index

In [8]:
ratings = ratings[ratings['User-ID'].isin(y)]

In [9]:
ratings_with_books = ratings.merge(books,on = "ISBN")

In [10]:
num_rating = ratings_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()

In [11]:
num_rating.rename(columns={"Book-Rating" : "No. of Ratings"},inplace=True)

In [12]:
final_rating = ratings_with_books.merge(num_rating,on='Book-Title')

In [13]:
final_rating = final_rating[final_rating['No. of Ratings']>=50]

In [14]:
final_rating.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,No. of Ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [15]:
final_rating.drop_duplicates(['User-ID','Book-Title'],inplace=True)

In [16]:
book_pivot = final_rating.pivot_table(columns='User-ID',index='Book-Title',values = 'Book-Rating')

In [17]:
book_pivot.fillna(0,inplace=True)

In [18]:
from scipy.sparse import csr_matrix

In [19]:
book_sparse = csr_matrix(book_pivot)

In [20]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [21]:
model.fit(book_pivot)

NearestNeighbors(algorithm='brute')

In [22]:
books_name = book_pivot.index

In [23]:
import pickle
pickle.dump(model,open('Articrafts/model.pkl','wb'))
pickle.dump(books_name,open('Articrafts/books_name.pkl','wb'))
pickle.dump(final_rating,open('Articrafts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('Articrafts/book_pivot.pkl','wb'))

In [24]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance,suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=10)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [26]:
book_name = 'A Bend in the Road'
recommend_book(book_name)

A Bend in the Road
Exclusive
The Cradle Will Fall
No Safe Place
Family Album
Last Man Standing
Lake Wobegon days
Long After Midnight
Abduction
Jacob Have I Loved
